In [ ]:
%run ../lib/kops.py
%run ../lib/e2e.py
%run ../lib/state.py

state = load_state()
state

In [ ]:
update_state(state, {"kops_create_cluster_spec": { "networking": "cilium" }})
state

In [ ]:
if not "project" in state:
    p = os.environ.get("PROJECT_ID")
    if not p:
        p = gcloud.current_project()
    update_state(state, { "project": p })

In [ ]:
if not "kops_version" in state:
    update_state(state, { "kops_version": get_kops_ci_latest() })

In [ ]:
if not "k8s_version" in state:
    update_state(state, { "k8s_version": get_kubernetes_version("stable-1.17") })

In [ ]:
create_cluster_spec_defaults = { "name": "foo.k8s.local", "project": state["project"], "zones": ["us-central1-f"], "cluster": { "spec": { "nodePortAccess": "0.0.0.0/0" } } }
update_state(state, {"kops_create_cluster_spec": create_cluster_spec_defaults})


In [ ]:
if not "results" in state:
    update_state(state, {"results": [] })
update_state(state, {"schema": "e2e.kops.k8s.io/v1"})

In [ ]:
save_state(state)
state

In [ ]:
e2e = download_kubetest(state["k8s_version"])

In [ ]:
kops = download_kops(state["kops_version"])
kops.version()

In [ ]:
cluster_name = state["kops_create_cluster_spec"]["name"]

In [ ]:
c = kops.create_cluster(state["kops_create_cluster_spec"])

In [ ]:
c = kops.cluster(cluster_name)

In [ ]:
c

In [ ]:
update_state(state, { "kops_spec": c.objects()})
save_state(state)

In [ ]:
c.update()

In [ ]:
c.wait()

In [ ]:
# a quick test for development: results = e2e.run(focus="Secrets should be consumable via the")
results = e2e.run("\[Conformance\]", skip="\[Serial\]", parallel=25)

In [ ]:
results

In [ ]:
tr = {"test": "e2e", "parallel": 25, "pass": len(results.failed_tests) == 0, "tests": []}
tr["tests"].extend(results.failed_tests)
tr["tests"].extend(results.passed_tests)
tr
state["results"].append(tr)

In [ ]:
save_state(state)
state

In [ ]:
c.delete()